## EDA for Code Similarity Determine

In [195]:
import os
import numpy as np
import pandas as pd
import torch
import random
import re

from tqdm import tqdm
from itertools import combinations
from transformers import AutoTokenizer, BertForSequenceClassification
# from transformers import AutoModel, AutoModelForSequenceClassification
# from datasets import load_dataset, load_metric
from rank_bm25 import BM25Okapi
from sklearn.model_selection import train_test_split

In [8]:
code_folder = "data/code/"
problem_folders = os.listdir(code_folder)

#### 원래 preprocess 함수

In [187]:
# PREPROCESSING FOR CODE SCRIPT
def preprocess_script(script):
    with open(script,'r',encoding='utf-8') as file:
        lines = file.readlines()
        preprocess_lines = []
        for line in lines:
            if line.lstrip().startswith('#'): # 주석으로 시작되는 행 skip
                continue
            line = line.rstrip()
            if '#' in line:
                line = line[:line.index('#')] # 주석 전까지 코드만 저장
            line = line.replace('\n','') # 개행 문자를 모두 삭제함
            line = line.replace('    ','\t') # 공백 4칸을 tab으로 변환
            
            if line == '': # 전처리 후 빈 라인은 skip
                continue
            
            preprocess_lines.append(line)
        preprocessed_script = '\n'.join(preprocess_lines) # 개행 문자로 합침
        
    return preprocessed_script

#### 내가 만든거

In [132]:
# PREPROCESSING FOR CODE SCRIPT
def preprocess_script(script):
    with open(script,'r',encoding='utf-8') as file:
        lines = file.readlines()
        preprocess_lines = []
        multiple_comments = False
        for line in lines:
            if line.lstrip().startswith('#'): # 주석으로 시작되는 행 skip
                continue
            if line.strip() == '"""' or line.strip() == "'''": # """나 '''로 시작되는 행이면 다중 주석이므로 skip
                if multiple_comments:
                    multiple_comments = False
                    continue
                multiple_comments = True
                continue
                
            if multiple_comments: # 다중 주석인 동안은 계속 skip
                continue
                
            # if line.lstrip().startswith('"') or line.lstrip().startswith("'"): # "나 '로 시작되는 행 skip
            #     continue
            
            line = line.rstrip()
            if '#' in line:
                line = line[:line.index('#')] # 주석 전까지 코드만 저장
            line = line.replace('\n','') # 개행 문자를 모두 삭제함
            line = line.replace('    ','\t') # 공백 4칸을 tab으로 변환
            # line = re.sub('("""[\w\W]*?""")', '', line) # """ """로 주석 처리된 문장 삭제
            # line = re.sub("('''[\w\W]*?''')", '', line)
            
            if line == '': # 전처리 후 빈 라인은 skip
                continue
            
            preprocess_lines.append(line)
        preprocessed_script = '\n'.join(preprocess_lines) # 개행 문자로 합침
        
    return preprocessed_script

In [192]:
preprocess_scripts = []
problem_nums = []

# 300개 Sample code에 대한 전처리
for problem_folder in tqdm(problem_folders):
    scripts = os.listdir(os.path.join(code_folder, problem_folder))
    problem_num = problem_folder
    for script in scripts:
        script_file = os.path.join(code_folder,problem_folder,script)
        preprocessed_script = preprocess_script(script_file)

        preprocess_scripts.append(preprocessed_script)
    # 번호 목록을 만들어서 전처리한 dataframe에 함께 넣어줌
    problem_nums.extend([problem_num]*len(scripts))

100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [01:48<00:00,  2.78it/s]


In [193]:
df = pd.DataFrame(data= {'code':preprocess_scripts, 'problem_num':problem_nums})

Clear Problem Number  
1,

In [194]:
df.head()

,code,problem_num
0,"M = 9\nN = 9\ndef main():\n\tfor i in range(1,...",problem001
1,"list1 = range(1,10)\nlist2 = list1\nfor i in l...",problem001
2,"for x in range(1, 10):\n\tfor y in range(1, 10...",problem001
3,"for i in range(1,10):\n\tfor n in range(1, 10)...",problem001
4,"for i in range(1,10):\n\tfor k in range(1,10):...",problem001


In [162]:
codes = df['code'].to_list()
# ndf = df[df['problem_num'] == 'problem002']['code'].to_list()

new_codes = []
for code in tqdm(codes):
    new_code = re.sub('("""[\w\W]*?""")', '<str>', code)
    if new_code == code:
        new_code = re.sub("('''[\w\W]*?''')", '<str>', code)
    new_codes.append(new_code)

100%|████████████████████████████████████████████████████████████████████████| 45101/45101 [00:00<00:00, 358776.28it/s]


In [163]:
ndf = df
ndf['code'] = new_codes
ndf.head()

,code,problem_num
0,"M = 9\nN = 9\ndef main():\n\tfor i in range(1,...",problem001
1,"list1 = range(1,10)\nlist2 = list1\nfor i in l...",problem001
2,"for x in range(1, 10):\n\tfor y in range(1, 10...",problem001
3,"for i in range(1,10):\n\tfor n in range(1, 10)...",problem001
4,"for i in range(1,10):\n\tfor k in range(1,10):...",problem001


In [176]:
problems = []
for code, problem in zip(ndf['code'].to_list(), ndf['problem_num'].to_list()):
    if 'url' in code and 'http' in code:
        problems.append(problem)
print(problems)

['problem149', 'problem211', 'problem214']


In [131]:
# ndf = df[df['problem_num'] == 'problem002']['code'].to_list()
# idx = 1

# for code in ndf:
#     print(f"{idx}th codes")
#     print(f"{code}")
#     print('-'*100)
#     idx += 1

1th codes
arr = list([])
for i in range(10):
	arr.append(int(input()))
arr.sort(reverse=True)
for i in range(3):
	print arr[i]
----------------------------------------------------------------------------------------------------
2th codes
mt = []
for i in range(10):
		mt.append(int(input()))
mt.sort()
print( mt[9] )
print( mt[8] )
print( mt[7] )
----------------------------------------------------------------------------------------------------
3th codes
order=[]
for number in range(10):
	hight=int(input())
	order.append(hight)
for j in range(9):
	for i in range(9):
		if order[i] < order[i+1]:
			a = order[i]
			b = order[i+1]
			order[i] = b
			order[i+1] = a
for i in range(3):
	print(order[i])
----------------------------------------------------------------------------------------------------
4th codes
j = []
for i in range(10) :
	j.append(int(input()))
j.sort(reverse = True)
for k in range(3) :
	print(j[k])
-----------------------------------------------------------------------------

In [51]:
# AutoTokenizer로 graphcodebert 사용하도록 설정
tokenizer = AutoTokenizer.from_pretrained("microsoft/graphcodebert-base")
# tokenizer.truncation_side = 'left'

df['tokens'] = df['code'].apply(tokenizer.tokenize) # Sample code를 Tokenization해서 tokens 컬럼에 추가
df['len'] = df['tokens'].apply(len) # tokens의 길이를 len 컬럼에 추가

Token indices sequence length is longer than the specified maximum sequence length for this model (773 > 512). Running this sequence through the model will result in indexing errors


In [53]:
df.describe()

,len
count,45101.000000
mean,160.123789
std,500.930345
min,5.000000
25%,61.000000
50%,108.000000
75%,200.000000
max,97566.000000


In [96]:
longdf = df[df['len'] >= 2000]
print(len(longdf))

lldf = longdf[longdf['len'] <= 20000]
print(len(lldf))

for idx, (code, problem) in enumerate(zip(lldf['code'], lldf['problem_num']), start=1):
    print(f"************{problem}************, code length = {len(code)}")
    print(f"{code}")
    print('-'*100)
    print()

14
13
************problem037************, code length = 4605
import tempfile, os, subprocess, sys, base64, time, zlib
(_, file) = tempfile.mkstemp()
f = open(file, "wb")
f.write(zlib.decompress(base64.b64decode(b"""eNrtWn1sW9UVv3biJG2pbWhaQlMaFxKWMMWk6cfSQMFO6/QFhS6QGqp2IXViJ3Hl2Jn93LqFdJ3SIr2ZDv8xpPAHrIwKsQ9GkQaKqi1NG2iKxKaARsVExSKUbnZTRAahmDapd+6759rXj6TtNGn7Jzdyzju/83Hvuffc++579/3E0Viv1+kIL3ryIKFcVZ5N5W2IW+5KqwBWQxbA/xWkmOQBbxD0bMSWRZPomtMC1MuBXy78avSMr9HbsugK1ONUJ1ADEYsti/6ikGRRaHnajrbVvJKh5pW7sugUtsOSk22nR7tytCtHfU5HsWGjmvhy8bcN/W7DuDjdjHqbBX1ami7IbnqdNGG/mWxZdBz1xjV2j4JdHrn5Ykb6GNY3V79MYlyc8nG4z+dtW7/2Pp+70uf1hyOVkZr1levXWkMBa7XaJjPqbtnqzOrHImxzIeYAlW9IGTqfP/1l9Sdvvf7iEume19dJ+++m+guEcVMvdGY9xRbCr+LOZ1ds+DK6Yq74aL/eNgt++xz4A3Pgj6czIbssmkO/ZA78tjnwdXPg34PfreQOYl61KyuvrIgXaXAC49FOu3896ejwhUNdxBPxyqTTI7d3uYLEF/B37u7uISHZHQjLlHiCQdKxN+iVPaSjJ+j1yx2ktZU6aQ3JrqDc2u3y+gHp7A74EWklWxob6ja1VlurrevYmOrVcdULfzr1L5Nf4eXeBVTDg3yyhLU7T8hdNd8wHgOOOy9FAr5QxC0MzxdyUp21Aq4X8HIBzxHwKgEX21Mj4OJ6YxNwc